1) This model uses default hyper-parameters for the logistic regression model

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [33]:
path = r"/content/drive/My Drive/Colab Notebooks/DataHack - Analytics Vidhya /JantaHack: Machine Learning in Agriculture"
train = pd.read_csv(path + "/train.csv")
test = pd.read_csv(path + "/test.csv")
submission = pd.read_csv(path + "/sample_submission.csv")


In [5]:
train.shape, test.shape, submission.shape

((88858, 10), (59310, 9), (59310, 2))

**Filling the null values in Number_Weeks_Used column**

In [34]:
train['Number_Weeks_Used'] = train['Number_Weeks_Used'].fillna(train.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].transform('median'))
test['Number_Weeks_Used'] = test['Number_Weeks_Used'].fillna(test.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].transform('median'))

# Data Preprocessing

In [35]:
training_labels = train.iloc[:, -1]
X_train = train.iloc[:, 1:-1]
X_test = test.iloc[:, 1:]

In [36]:
X_train.shape, X_test.shape

((88858, 8), (59310, 8))

In [37]:
data = pd.concat([X_train, X_test])

**OneHotEncoding for "Pesticide_Use_Category" and "Season"**

In [38]:
columns_names_encod = data.columns[[3, 7]]
data = pd.get_dummies(data, columns=columns_names_encod)

In [39]:
data.shape

(148168, 12)

**Splitting the data back**

In [41]:
final_train = data[:train.shape[0]]
final_test = data[train.shape[0]:]

In [42]:
final_train.shape, final_test.shape

((88858, 12), (59310, 12))

**Scaling the numerical variables**

In [43]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

final_train.iloc[:, [0, 3, 4, 5]] = sc.fit_transform(final_train.iloc[:, [0, 3, 4, 5]])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [45]:
final_test.iloc[:, [0, 3, 4, 5]] = sc.transform(final_test.iloc[:, [0, 3, 4, 5]])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# Creating cross validation set

In [46]:
training_labels.value_counts()

0    74238
1    12307
2     2313
Name: Crop_Damage, dtype: int64

In [47]:
from sklearn.model_selection import train_test_split
X_training, X_cv, y_training, y_cv = train_test_split(final_train, training_labels, test_size =0.2, random_state=21)

# Training model on training dataset


In [49]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=500)

clf.fit(X_training, y_training)
y_training_pred = clf.predict(X_training)
y_cv_pred = clf.predict(X_cv)

**Checking accuracy score for training dataset and cross validation set**

In [51]:
from sklearn.metrics import accuracy_score
training_accuracy_score = accuracy_score(y_training, y_training_pred)
cv_accuracy_score = accuracy_score(y_cv, y_cv_pred)

print(training_accuracy_score)
print(cv_accuracy_score)

0.8393635877669302
0.8364280891289669


# Testing the model on test data

In [54]:
predictions_test = clf.predict(final_test)
predictions_test = pd.Series(predictions_test)

frame = {'ID': test.ID, 
         'Crop_Damage': predictions_test}
submission = pd.DataFrame(frame)
submission.to_csv(path + '/Logistic_Regression_1_submission.csv', index=False)